In [1]:
import json
import sys

sys.path.append("/Users/personal/Desktop/scriptie")
sys.path.append("/Users/personal/Desktop/scriptie/src")
sys.path.append("/Users/personal/Desktop/scriptie/src/backend")
from src.backend.LlmClient import MlxLlama

/Users/personal/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = MlxLlama(model_name="mlx-community/Meta-Llama-3-8B-Instruct-8bit")

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 92327.45it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
def get_transcript(path):
    with open(path, "r") as f:
        data = json.load(f)

    return data["text"]


def get_transcript_times(path):
    with open(path, "r") as f:
        data = json.load(f)

    filtered = [{"text": d["text"], "start": d["start"]} for d in data["segments"]]

    return filtered


def split_transcript(transcript, chunk_size=10000):
    return [
        transcript[i : i + chunk_size] for i in range(0, len(transcript), chunk_size)
    ]


def get_points(text):
    resp = llm.run(
        [
            {
                "role": "system",
                "content": "Je bent een behulpzame assistent die van een gegeven stuk tekst de hoofd onderwerpen geeft, gescheiden met een ';'. Geef alleen de aller belangrijkste punten mee. Geef enkel en alleen de punten, gescheiden met ';' mee, geen extra informatie of tekst.",
            },
            {"role": "user", "content": text},
        ]
    )

    return resp

In [27]:
def generate_agenda(points):
    return [{"agendaPoint": p.replace("\n\n", "").strip()} for p in points if not p.strip().startswith("Hier")]


def clear_dups(agenda):
    dict_tuples = [tuple(sorted(d.items())) for d in agenda]
    unique_dict_tuples = set(dict_tuples)
    unique_agenda = [dict(t) for t in unique_dict_tuples]

    return unique_agenda


def get_agendapoint_start(point, path, offset=0):
    i = 0
    for sentence in get_transcript_times(path):
        # if i < offset:
        #     i += 1
        #     continue

        resp = llm.run(
            [
                {
                    "role": "system",
                    "content": "Je bent een behulpzame assistent die van een gegeven stuk tekst een agenda punt, aan geeft of dit agenda punt begint bij de tekst. Reageer alleen 'ja' als het overeenkomt, en 'nee' als dit niet zo is.",
                },
                {
                    "role": "user",
                    "content": f"agenda punt: {point}\n\nText: {sentence['text']}",
                },
            ]
        )
        print("tekt", sentence["text"], "punt", point, "antwoord:", resp)
        if resp.strip() == "ja":
            return offset, sentence["start"]
        i += 1

    return 0, -1


def get_agenda_times(agenda, filepath):
    new_agenda = []

    offset = 0
    for point in agenda:
        offset, start = get_agendapoint_start(point["agendaPoint"], filepath, offset)
        if start != -1:
            new_agenda.append({"agendaPoint": point["agendaPoint"], "time": start})

    return new_agenda

In [28]:
path = "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2022/transcripts/919518.mp4.json"

transcript = get_transcript(path)

agenda = []

for sub in split_transcript(transcript):
    points = get_points(sub)
    print(points)
    print("")
    agenda += generate_agenda(points.split(";"))
    break

print(get_agenda_times(agenda, path))

Hier zijn de hoofdonderwerpen, gescheiden met een ';':

Bestemmingsplan; verkeersveiligheid; gemeente; Numansdorp; burgemeester De Zeelstraat; woningen; fietspad; verkeersbewegingen; griffie; zienswijze; inspraak.

tekt  Goedendag allemaal. punt ':Bestemmingsplan antwoord: nee
tekt  Maar goed, het is goed. punt ':Bestemmingsplan antwoord: nee
tekt  Dames en heren, het is op mijn klokje half acht en we gaan beginnen. punt ':Bestemmingsplan antwoord: nee
tekt  Aan onze eerste beeldvormende vergadering van het jaar 2022. punt ':Bestemmingsplan antwoord: nee
tekt  Ik ga toch van de gelegenheid gebruik maken om vanuit deze hoek samen met Leanne die naast me zit punt ':Bestemmingsplan antwoord: nee
tekt  alle een heel goed 2022 te wensen. punt ':Bestemmingsplan antwoord: nee
tekt  En het mag duidelijk zijn dat 2022 voor ons allen best een spannend jaar gaat worden, punt ':Bestemmingsplan antwoord: nee
tekt  want het is per slotverrekening een verkiezingsjaar. punt ':Bestemmingsplan antwoord: